# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
## Loading the model

In [2]:
from datasets import load_dataset

#load dataset
ds = load_dataset("glue", 'sst2', split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=44
)
ds

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 872/872 [00:00<00:00, 376860.70 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 53879
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 13470
    })
})

In [3]:
splits = ["train", "test"]
train_ds = ds["train"]
test_ds = ds["test"]

In [4]:
# Get a shorter dataset for the example
for split in splits:
    ds[split] = ds[split].shuffle(seed=44).select(range(600))

In [5]:
ds["train"][0]

{'sentence': 'it introduces you to new , fervently held ideas and fanciful thinkers ',
 'label': 1,
 'idx': 10006}

In [6]:
# Pre pocess dataset
from transformers import AutoTokenizer #GPT2Tokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["sentence"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 51.0kB/s]
config.json: 100%|██████████| 665/665 [00:00<00:00, 4.23MB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 22.3MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 24.6MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 60.4MB/s]
Map: 100%|██████████| 600/600 [00:00<00:00, 13322.86 examples/s]


Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 600
})

In [7]:
# Load and setup the model
from transformers import AutoModelForSequenceClassification

# a classification label, with possible values including negative (0), positive (1).
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.requires_grad = True

model.safetensors: 100%|██████████| 548M/548M [00:02<00:00, 211MB/s] 
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [9]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

tokenizer.pad_token = tokenizer.eos_token

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# More about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/results",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.971100,1.307386,0.803333
2,0.939400,1.113263,0.845000


TrainOutput(global_step=1200, training_loss=1.3215505218505859, metrics={'train_runtime': 150.4161, 'train_samples_per_second': 7.978, 'train_steps_per_second': 7.978, 'total_flos': 7709274832896.0, 'train_loss': 1.3215505218505859, 'epoch': 2.0})

In [10]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 1.1132631301879883,
 'eval_accuracy': 0.845,
 'eval_runtime': 9.5495,
 'eval_samples_per_second': 62.83,
 'eval_steps_per_second': 62.83,
 'epoch': 2.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
# Load and set up the model

from peft import LoraConfig, TaskType
config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

from transformers import AutoModelForSequenceClassification #AutoModelForCausalLM
model = AutoModelForSequenceClassification.from_pretrained("gpt2")

from peft import get_peft_model, LoraModel
lora_model = get_peft_model(model, config)
lora_model.config.pad_token_id = model.config.eos_token_id


model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
lora_model.print_trainable_parameters()

trainable params: 814,080 || all params: 125,253,888 || trainable%: 0.6499438963523432


In [13]:
print(lora_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

# Use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["sentence"], truncation=True, padding=True), batched=True
    )
    
# PEFT LoRA expects labels not label on training
tokenized_dataset["train"] = tokenized_dataset["train"].rename_column("label", "labels")
tokenized_dataset["test"] = tokenized_dataset["test"].rename_column("label", "labels")

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map: 100%|██████████| 600/600 [00:00<00:00, 7898.28 examples/s]


Dataset({
    features: ['sentence', 'labels', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 600
})

In [15]:
tokenized_dataset['train'].set_format('torch', columns=['labels', 'input_ids', 'attention_mask'])
tokenized_dataset['test'].set_format('torch', columns=['labels', 'input_ids', 'attention_mask'])
tokenized_dataset["train"]

Dataset({
    features: ['sentence', 'labels', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 600
})

In [16]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [17]:
# Train the model
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# More about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/results_lora",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
        remove_unused_columns=False
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.501100,0.999212,0.466667
2,0.963200,0.713286,0.503333


TrainOutput(global_step=1200, training_loss=1.165559679667155, metrics={'train_runtime': 126.9019, 'train_samples_per_second': 9.456, 'train_steps_per_second': 9.456, 'total_flos': 34004551680000.0, 'train_loss': 1.165559679667155, 'epoch': 2.0})

In [18]:
# Save the model
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [19]:
# Inference with PEFT
from peft import AutoPeftModelForSequenceClassification, get_peft_config, PeftConfig

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora", ignore_mismatched_sizes=True)
lora_model.print_trainable_parameters()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,072 || all params: 125,253,888 || trainable%: 0.002452618476801295


In [20]:
from transformers import AutoTokenizer, Trainer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [21]:
from transformers import AutoTokenizer, Trainer, DataCollatorWithPadding, TrainingArguments

trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/results_lora",
        learning_rate=2e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics,
)

In [22]:
# Evaluating the model


In [23]:
trainer.evaluate()  #tokenized_dataset["test"])

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6939945220947266,
 'eval_accuracy': 0.5416666666666666,
 'eval_runtime': 15.4316,
 'eval_samples_per_second': 38.881,
 'eval_steps_per_second': 38.881}